In [0]:
#Download the required libraries
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.onnx
import torchvision

%matplotlib inline

plt.ion()   # interactive mode

**ЗАГРУЗКА ДАННЫХ**

In [0]:
#Получаю адрес папки /content (папка диска)
dir = os.path.abspath(os.curdir)
# указываем путь до датасета 
data_dir=os.path.join(dir, "/content/drive/My Drive/DATA")

# Масштабируем данные до 224*224 и нормализуем для train и test
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

#Создаю функцию для загрузки изображений из датасета и ресайза для модели Create function to get your(my) images dataset and resize it to size for model
def get_dataset(data_dir, data_transforms):

    # Создаю train и test датасеты
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'test']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                                 shuffle=True, num_workers=4)
                  for x in ['train', 'test']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
    #Получаем классы из имен папок train датасета
    classes = image_datasets['train'].classes

    return dataloaders["train"], dataloaders['test'], classes, dataset_sizes

In [3]:
# Загружаем данные train и test
trainloader, testloader, classes, dataset_sizes=get_dataset(data_dir,data_transforms)
print('Классов: ',  classes)
print('Набор данных содержит: ',  dataset_sizes ," снимков")

Классов:  ['0', '1']
Набор данных содержит:  {'train': 1331, 'test': 378}  снимков


**ПРЕДПОДГОТОВКА ИЗОБРАЖЕНИЙ**

In [0]:
# Функция для создания нового изображения после трансформации 
def imshow_DenseNet(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # пауза для обновления графика

**СОЗДАНИЕ МОДЕЛИ**

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Каждая эпоха имеет свою фазу обучения и проверки
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Перебираем данные
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Обнуляем градиент
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + оптимизатор только в тренировочной фазе
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # статистика
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
dataloaders = {'train': trainloader, 'test': testloader}
# model_ft = models.resnet18(pretrained=True) - было
model_ft = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)
num_ftrs = model_ft.classifier.in_features
# Здесь размер каждого выходного образца устанавливается равным числу классов.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, 256), 
                                    nn.ReLU(), 
                                    nn.Dropout(0.4), 
                                    nn.Linear(256, len(classes))) 

device = 'cuda'
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Оптимизируем все параметры
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Снижение LR каждые 7 эпох 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


**ОБУЧЕНИЕ МОДЕЛИ**

In [7]:
#Train the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

Epoch 0/4
----------
train Loss: 0.2616 Acc: 0.9264
test Loss: 0.2074 Acc: 0.9392

Epoch 1/4
----------
train Loss: 0.2450 Acc: 0.9294
test Loss: 0.1883 Acc: 0.9392

Epoch 2/4
----------
train Loss: 0.2212 Acc: 0.9294
test Loss: 0.2103 Acc: 0.9392

Epoch 3/4
----------
train Loss: 0.2213 Acc: 0.9294
test Loss: 0.1791 Acc: 0.9392

Epoch 4/4
----------
train Loss: 0.2070 Acc: 0.9301
test Loss: 0.1787 Acc: 0.9392

Training complete in 38m 21s
Best val Acc: 0.939153


**РЕЗУЛЬТАТЫ МОДЕЛИ**

In [8]:
# Визуализируем результаты
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        
        for printdata in list(zip(predicted,labels,outputs)):
            printclass =[classes[int(printdata[0])],classes[int(printdata[1])]]
            print('Predict class - {0}, real class - {1}, probability ({2},{3}) - {4}'.format( printclass[0],printclass[1],
                                                                              classes[0], classes [1],printdata[2]))

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # imshow_DenseNet(torchvision.utils.make_grid(images))
        #print('GroundTruth: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))
print('Accuracy of the network on the', dataset_sizes['test'], 'test images: %d %%' % (
    100 * correct / total))

Predict class - 1, real class - 1, probability (0,1) - tensor([-1.1853,  0.9615], device='cuda:0')
Predict class - 1, real class - 1, probability (0,1) - tensor([-1.1404,  0.8346], device='cuda:0')
Predict class - 1, real class - 0, probability (0,1) - tensor([-1.3528,  0.8593], device='cuda:0')
Predict class - 1, real class - 1, probability (0,1) - tensor([-0.9753,  0.8338], device='cuda:0')
Predict class - 1, real class - 1, probability (0,1) - tensor([-1.2367,  0.9193], device='cuda:0')
Predict class - 1, real class - 1, probability (0,1) - tensor([-1.1719,  0.8854], device='cuda:0')
Predict class - 1, real class - 1, probability (0,1) - tensor([-1.6000,  1.2863], device='cuda:0')
Predict class - 1, real class - 1, probability (0,1) - tensor([-1.0873,  0.9156], device='cuda:0')
Predict class - 1, real class - 1, probability (0,1) - tensor([-1.4439,  1.0994], device='cuda:0')
Predict class - 1, real class - 1, probability (0,1) - tensor([-1.2784,  0.8068], device='cuda:0')
Predict cl

**СОХРАНЯЕМ МОДЕЛЬ**

In [0]:
# Сохраним обученную модель
dir='/content/drive/My Drive/MODELS CT/'
PATH =os.path.join(dir, "model v0.1")
torch.save(model_ft, PATH)

**ИСПОЛЬЗУЕМ ОБУЧЕННУЮ МОДЕЛЬ ДЛЯ ПРЕДСКАЗАНИЙ НА РАБОЧИХ ДАННЫХ**

In [0]:
modelBR=torch.load('/content/drive/My Drive/MODELS CT/model v0.1(CT)')

In [0]:
dir='/content/drive/My Drive/MODELS CT/'
PATH = os.path.join(dir, "model.pt")
torch.save(modelBR.state_dict(), PATH)

Загружаем и преобразуем изображение в модель

In [0]:
path_file = '/content/drive/My Drive/MODELS CT/fulltestdata/'
transforms_file = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
image_datasets = datasets.ImageFolder(os.path.join(path_file,'test'), transforms_file)
workloader2 = torch.utils.data.DataLoader(image_datasets, batch_size=1, shuffle=True, num_workers=4)                                          

In [0]:
# Предсказываем класс
classes_test = ['0', '1', '2', '3', '4', '5']
with torch.no_grad():
    for data in workloader2:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = modelBR(images)
        _, predicted = torch.max(outputs.data, 1)
        for printdata in list(zip(predicted,labels,outputs)):
            printclass =[classes_test[int(printdata[0])],classes_test[int(printdata[1])]]
            print(u'Предсказанный класс по классификации BI-RADS - {0}'.format(printclass[0]))

Предсказанный класс по классификации BI-RADS - 4
